In [15]:
import hoomd
import gsd.hoomd
import numpy
import numpy as np
import math
import itertools
import matplotlib.pyplot as plt

In [16]:
a = gsd.hoomd.Snapshot()
a.particles.N = 2
a.particles.types = ['A']
a.particles.typeid = [0,0]
a.particles.position = [[1,1,2], [1,2,1]]
a.configuration.box = [8, 8, 8, 0, 0, 0]
a.particles.mass = [2] * 2

In [17]:
#Script in order to account for Intertia, which hopefully solves the line 9 problem of providing no torques
#The problem: adding the mass property to the particles breaks the line 4 command
general_positions = numpy.array([[0, 0, 0], [1, 0, 0]])

I_ref = numpy.array([[0.4, 0, 0],
                   [0, 0.4, 0],
                   [0, 0, 0.4]])
I_general = numpy.zeros(shape=(3,3))
for r in general_positions:
    I_general += I_ref + 1 * (numpy.dot(r, r) * numpy.identity(3) - numpy.outer(r, r))
    
I_diagonal, E_vec = numpy.linalg.eig(I_general)

R = E_vec.T

diagonal_positions = numpy.dot(R, general_positions.T).T

I_check = numpy.zeros(shape=(3,3))
for r in diagonal_positions:
    I_check += I_ref + 1 * (numpy.dot(r, r) * numpy.identity(3) - numpy.outer(r, r))

#mass = 1
#I = numpy.zeros(shape=(3,3))
#for r in central_position:
#    I += mass * (numpy.dot(r, r) * numpy.identity(3) - numpy.outer(r, r))
I_check

array([[0.8, 0. , 0. ],
       [0. , 1.8, 0. ],
       [0. , 0. , 1.8]])

In [18]:
a.particles.moment_inertia = [I_general[0, 0], I_general[1, 1], I_general[2, 2]] * 2
print(a.particles.moment_inertia[:])

[0.8, 1.7999999999999998, 1.7999999999999998, 0.8, 1.7999999999999998, 1.7999999999999998]


In [19]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)
sim.create_state_from_snapshot(a)

In [20]:
rigid = hoomd.md.constrain.Rigid()

rigid.body['dimer'] = {
    "constituent_types": ['A', 'A'],
    "positions": [[0, 0, -0.5],[0,0,0.5]],
    "charges": [0.0, 0.0],
    "diameters": [1.0, 1.0]
}

In [21]:
integrator = hoomd.md.Integrator(dt=0.005, integrate_rotational_dof=True)

cell=hoomd.md.nlist.Cell(buffer=0.4)

lperp=0.3
lpar=1.0
sigmin=2*min(lperp,lpar)
sigmax=2*max(lperp,lpar)

lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=10, sigma=1)
lj.r_cut[('A', 'A')] = 2.5

In [22]:
sim.operations.integrator=integrator

In [23]:
integrator.rigid=rigid

In [24]:
rigid_centers_and_free=hoomd.filter.Rigid(("center","free"))
nve=hoomd.md.methods.NVE(filter=hoomd.filter.All())
integrator.methods.append(nve)
integrator.forces.append(lj)

In [25]:
sim.run(0)

In [26]:
g=sim.state.get_snapshot()
g.particles.orientation

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [27]:
thermodynamic_properties=hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
gsd_writer=hoomd.write.GSD(filename='traj_different_directions2.gsd',trigger=hoomd.trigger.Periodic(1),mode='wb',filter=hoomd.filter.All())
sim.operations.writers.append(gsd_writer)

In [28]:
sim.operations.computes.append(thermodynamic_properties)
sim.run(10000)